In [ ]:
from nustar_gen import wrappers, info
import os

# NOTE!!!!
#### The code below is an *example* only for how to process straylight data for analysis.

### IT WILL NOT WORK WITH THE EXAMPLE DATA IN THIS REPOSITORY

#### Please use it as a template only for your own analysis.


### How to make images in DET1 coordinates

In [ ]:
# top == some path to your top level data directory. 

seqid='10202005004'
obs = info.Observation(seqid=seqid)
obs = info.Observation(path=f'{top}/data/', seqid=seqid)

print(obs._evtfiles['B'][0])
# Spawns an Xselect instance behind the scenes. Output in the same place as the event data
det1B_file = wrappers.make_det1_image(obs._evtfiles['B'][0],elow=3, ehigh=20)
print(det1B_file)


### How to make lightcurves from DET1 data
1. Go to the DET1 image produced above and make a region that covers the straylight that you want to analyze. Note that this can be a large circle with lots of area outside of the FoV. i.e.:

<img src="./example_data/det1_image_example.png" alt="Drawing" style="width: 800px;"/>

2. Provide the full path to the region file. **Make sure to save the file using IMAGE coordinates.** See the example image above.

3. Extract the events using Xselect:

In [ ]:
# Go make the region file using ds9. Make sure to to use 
reg_file = obs.path+'/'+obs.seqid+'/event_cl/srcB.reg'
filt_file = wrappers.extract_det1_events(obs._evtfiles['B'][0], regfile=reg_file)

# filt_file is now the full path to the extracted event file

4. Make a lightcurve script that spawns nuproducts. Note that we have turned off all of the nuproducts flags that cause it to apply the PSF, exposure, and vignetting corrections. This does do the livetime corrections. For straylight sources the motion of the telescope pointing/mast shouldn't affect the response of the instrument, and we'll assume that we'll take care of computing the effective area when doing spectroscopy. **Note:** this wrapper has a "barycorr" option, but it is currently disabled. If you're doing pulsar timing with stray light then you'll probably want to construct the nuproducts call yourself.

In [ ]:
from astropy import units as u
time_bin = 100*u.s
lc_script = wrappers.make_det1_lightcurve(filt_file, mod='B', elow=3, ehigh=10, time_bin=time_bin, outpath='./10202005004/products/')
# lc_script is now the path to the nuproducts script. Right now appears in the same location as the notebook.

5. Run the script right from the notebook if you'd like. We can use these notebook magic commands to set the environment variables so that nuproducts runs smoothly behind the scenes:

In [ ]:
import os

%env HEADASNOQUERY=
%env HEADASPROMPT=/dev/null
os.system('./'+lc_script)

# The lightcurve file and its associated .gif render will now be in the outpath location to be used for downstream analysis.